In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

%matplotlib inline

In [2]:
nltk.download("punkt") 

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [3]:
ps= PorterStemmer()
wnl = WordNetLemmatizer()

In [4]:
#TODO PREPROCESS ALL DOCUMENTS
#TODO 

In [5]:
data = pd.read_csv("../dataset/Train.csv")
data

,Text_ID,Product_Description,Product_Type,Sentiment
0,3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2
1,6254,RT @mention Line for iPad 2 is longer today th...,9,2
2,8212,Crazy that Apple is opening a temporary store ...,9,2
3,4422,The lesson from Google One Pass: In this digit...,9,2
4,5526,RT @mention At the panel: &quot;Your mom has a...,9,2
...,...,...,...,...
6359,2945,Thanks @mention for hosting this event #sxsw #...,9,2
6360,4528,"SXSW iPhone app is awesome, but iPad app crash...",7,1
6361,6526,RT @mention RT @mention Download 20+ free trac...,9,2
6362,2110,OH at Texas Social Media Awards: 'You don't ne...,9,2


In [6]:
sample_data = data['Product_Description'][0]

In [7]:
sample_data

'The Web Designer\x89Ûªs Guide to iOS (and Android) Apps, today @mention 10 a.m! {link} #sxsw'

##PREPROCESS DATA

In [8]:
data = data.drop(['Text_ID','Product_Type' ], axis=1)
data

,Product_Description,Sentiment
0,The Web DesignerÛªs Guide to iOS (and Android...,2
1,RT @mention Line for iPad 2 is longer today th...,2
2,Crazy that Apple is opening a temporary store ...,2
3,The lesson from Google One Pass: In this digit...,2
4,RT @mention At the panel: &quot;Your mom has a...,2
...,...,...
6359,Thanks @mention for hosting this event #sxsw #...,2
6360,"SXSW iPhone app is awesome, but iPad app crash...",1
6361,RT @mention RT @mention Download 20+ free trac...,2
6362,OH at Texas Social Media Awards: 'You don't ne...,2


In [15]:
def preprocess_data_lemmatize(text):
    cleaned_data = re.sub(r"[^A-Za-z]","", text )
    cleaned_data = cleaned_data.lower()
    cleaned_data = word_tokenize(cleaned_data)
    cleaned_data = [wnl.lemmatize(word) for word in cleaned_data if word not in stopwords.words('english')]
    cleaned_data =" ".join(cleaned_data)
    return cleaned_data

    

In [16]:
def preprocess_data_stem(text):
    cleaned_data = re.sub(r"[^A-Za-z ]","", text )
    print(cleaned_data)
    cleaned_data = cleaned_data.lower()
    cleaned_data = word_tokenize(cleaned_data)
    cleaned_data = [ps.stem(word) for word in cleaned_data if word not in stopwords.words('english')]
    cleaned_data =" ".join(cleaned_data)
    return cleaned_data

    